# Data Processing Notebook:

In [1]:
# Install any needed dependencies
!pip install pydub
!pip install librosa
!pip install regex
!pip install soundfile
!pip install noisereduce

     |████████████████████████████████| 213 kB 5.8 MB/s            
     |████████████████████████████████| 377 kB 3.7 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 323 kB 5.1 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for audioread: filename=audioread-2.1.9-py3-none-any.whl size=23154 sha256=0e3eaff32dac793fc38b94701aa7ff66e72e717e4da933b162d9e4f266840a29
  Stored in directory: /home/jovyan/.cache/pip/wheels/a2/a3/bd/ec1568ce7515115a11ab686d509ad302124c782af065de47ee
  Created wheel for resampy: filename=resampy-0.2.2-py3-none-any.whl size=320731 sha256=97d18990c936933c90e154f03e4101452fd7600674366716a694fc26bfe0f492
  Stored in directory: /home/jovyan/.cache/pip/wheels/86/2c/7d/46a32a246b0e5939cea2c5ec1492164073e0c5d16d666ae2cd
Successfully built audioread resampy
     |████████████████████████████████| 763 kB 2.9 MB/s            


In [2]:
import os
import sys
sys.path.append('../')

from functions import processing_funcs, utils
from pydub.utils import mediainfo

### Split stereo channels into mono:

In [3]:
%%time
voice_dir = os.path.join('..', 'voice_data')
mono_dir_path = os.path.join(voice_dir, 'mono_channels')
utils.make_dir(mono_dir_path)
processing_funcs.split_stereo_audio(voice_dir_path = '../voice_data/')

Creating directory at ../voice_data/mono_channels...


Splitting file 1 of 31 at ../voice_data/4175.wav...

Splitting file 2 of 31 at ../voice_data/4504.wav...

Splitting file 3 of 31 at ../voice_data/4708.wav...

Splitting file 4 of 31 at ../voice_data/4745.wav...

Splitting file 5 of 31 at ../voice_data/4823.wav...

Splitting file 6 of 31 at ../voice_data/4874.wav...

Splitting file 7 of 31 at ../voice_data/4889.wav...

Splitting file 8 of 31 at ../voice_data/4984.wav...

Splitting file 9 of 31 at ../voice_data/5000.wav...

Splitting file 10 of 31 at ../voice_data/5051.wav...

Splitting file 11 of 31 at ../voice_data/5220.wav...

Splitting file 12 of 31 at ../voice_data/5635.wav...

Splitting file 13 of 31 at ../voice_data/5926.wav...

Splitting file 14 of 31 at ../voice_data/6015.wav...

Splitting file 15 of 31 at ../voice_data/6062.wav...

Splitting file 16 of 31 at ../voice_data/6065.wav...

Splitting file 17 of 31 at ../voice_data/6093.wav...

Splitting file 18 of 31 at ../voice_

### All of our stereo audio should now be split into mono channels:

In [4]:
mono_dir = utils.read_dir_files(dir_path = mono_dir_path,
                                file_regex = r'[0-9]+\_(?:L|R)\.wav')

print('Total number of mono files:', len(mono_dir), '\n')

for mono_file in mono_dir:
    print(mono_file)


Total number of mono files: 62 

4175_L.wav
4175_R.wav
4504_L.wav
4504_R.wav
4708_L.wav
4708_R.wav
4745_L.wav
4745_R.wav
4823_L.wav
4823_R.wav
4874_L.wav
4874_R.wav
4889_L.wav
4889_R.wav
4984_L.wav
4984_R.wav
5000_L.wav
5000_R.wav
5051_L.wav
5051_R.wav
5220_L.wav
5220_R.wav
5635_L.wav
5635_R.wav
5926_L.wav
5926_R.wav
6015_L.wav
6015_R.wav
6062_L.wav
6062_R.wav
6065_L.wav
6065_R.wav
6093_L.wav
6093_R.wav
6126_L.wav
6126_R.wav
6157_L.wav
6157_R.wav
6193_L.wav
6193_R.wav
6239_L.wav
6239_R.wav
6255_L.wav
6255_R.wav
6278_L.wav
6278_R.wav
6372_L.wav
6372_R.wav
6379_L.wav
6379_R.wav
6476_L.wav
6476_R.wav
6862_L.wav
6862_R.wav
6869_L.wav
6869_R.wav
6899_L.wav
6899_R.wav
6938_L.wav
6938_R.wav
6952_L.wav
6952_R.wav


### We now have 62 mono files: 2 * 31 original files. This makes sense!

Now to check to make sure they look okay:

In [5]:
for file in mono_dir:
    file_path = os.path.join(mono_dir_path, file)
    file_info = mediainfo(file_path)
    print('File:', file_path, '|',
          'Number of Channels:', file_info['channels'], '|',
          'Sampling Rate:', file_info['sample_rate'], '|',
          'Duration (mins):', utils.get_file_duration(file_path))

File: ../voice_data/mono_channels/4175_L.wav | Number of Channels: 1 | Sampling Rate: 8000 | Duration (mins): 1800.000000
File: ../voice_data/mono_channels/4175_R.wav | Number of Channels: 1 | Sampling Rate: 8000 | Duration (mins): 1800.000000
File: ../voice_data/mono_channels/4504_L.wav | Number of Channels: 1 | Sampling Rate: 8000 | Duration (mins): 470.760000
File: ../voice_data/mono_channels/4504_R.wav | Number of Channels: 1 | Sampling Rate: 8000 | Duration (mins): 470.760000
File: ../voice_data/mono_channels/4708_L.wav | Number of Channels: 1 | Sampling Rate: 8000 | Duration (mins): 1800.000000
File: ../voice_data/mono_channels/4708_R.wav | Number of Channels: 1 | Sampling Rate: 8000 | Duration (mins): 1800.000000
File: ../voice_data/mono_channels/4745_L.wav | Number of Channels: 1 | Sampling Rate: 8000 | Duration (mins): 1425.450000
File: ../voice_data/mono_channels/4745_R.wav | Number of Channels: 1 | Sampling Rate: 8000 | Duration (mins): 1425.450000
File: ../voice_data/mono_c

### Channels and duration look good, and the native sampling rate was preserved. Perfect.

### Now to eliminate noise as much as possible from the mono files:

We'll use the 'noisereduce' package and see how that affects our data:

noisereduce package information here:
https://pypi.org/project/noisereduce/#:~:text=Noisereduce%20is%20a%20noise%20reduction,a%20form%20of%20Noise%20Gate.

We'll be using the simplest implementation to start out with and we can try out different methods later on.

@software{tim_sainburg_2019_3243139,
  author       = {Tim Sainburg},
  title        = {timsainb/noisereduce: v1.0},
  month        = jun,
  year         = 2019,
  publisher    = {Zenodo},
  version      = {db94fe2},
  doi          = {10.5281/zenodo.3243139},
  url          = {https://doi.org/10.5281/zenodo.3243139}
}


@article{sainburg2020finding,
  title={Finding, visualizing, and quantifying latent structure across diverse animal vocal repertoires},
  author={Sainburg, Tim and Thielk, Marvin and Gentner, Timothy Q},
  journal={PLoS computational biology},
  volume={16},
  number={10},
  pages={e1008228},
  year={2020},
  publisher={Public Library of Science}
}

In [6]:
%%time

processing_funcs.reduce_noise(voice_dir_path = voice_dir)


Removing noise from file 1 of 62 at ../voice_data/mono_channels/4175_L.wav...



  0%|          | 0/24 [00:00<?, ?it/s]

Removing noise from file 2 of 62 at ../voice_data/mono_channels/4175_R.wav...



  0%|          | 0/24 [00:00<?, ?it/s]

Removing noise from file 3 of 62 at ../voice_data/mono_channels/4504_L.wav...



...

Removing noise from file 61 of 62 at ../voice_data/mono_channels/6952_L.wav...



  0%|          | 0/24 [00:00<?, ?it/s]

Removing noise from file 62 of 62 at ../voice_data/mono_channels/6952_R.wav...



  0%|          | 0/24 [00:00<?, ?it/s]

Noise reduction complete.

CPU times: user 4min 28s, sys: 15.9 s, total: 4min 44s
Wall time: 4min 51s


### Now to remove silence from the audio files:

In [8]:
%%time
silence_dir_path = os.path.join(mono_dir_path, 'silence_removed')
utils.make_dir(silence_dir_path)
processing_funcs.remove_silence(mono_channel_dir = mono_dir_path)

Creating directory at ../voice_data/mono_channels/silence_removed...


Removing periods of silence from file 1 of 62...

Removing periods of silence from file 2 of 62...

Removing periods of silence from file 3 of 62...

Removing periods of silence from file 4 of 62...

Removing periods of silence from file 5 of 62...

Removing periods of silence from file 6 of 62...

Removing periods of silence from file 7 of 62...

Removing periods of silence from file 8 of 62...

Removing periods of silence from file 9 of 62...

Removing periods of silence from file 10 of 62...

Removing periods of silence from file 11 of 62...

Removing periods of silence from file 12 of 62...

Removing periods of silence from file 13 of 62...

Removing periods of silence from file 14 of 62...

Removing periods of silence from file 15 of 62...

Removing periods of silence from file 16 of 62...

Removing periods of silence from file 17 of 62...

Removing periods of silence from file 18 of 62...

Removing periods of 